In [1]:
%config Completer.use_jedi=False
import folium
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests,json
from geopy.geocoders import Nominatim
import time

In [2]:
# cairo district
cairo_url='https://en.wikipedia.org/wiki/Category:Districts_of_Cairo'
page=requests.get(url=cairo_url).text

In [3]:
soup=BeautifulSoup(page)

In [4]:
page_part=soup.find(name='div',attrs={"class":"mw-category"})

In [5]:
data=page_part.findAll('li')

In [6]:
cairo_dist=[]
for i in data:
    cairo_dist.append(i.text)

In [7]:
cairo_dist.pop(0)

'Template:Districts of Cairo'

In [8]:
print(cairo_dist)

['Abbassia', 'Ain Shams', 'Azbakeya', 'Bab al-Louq', 'Boulaq', 'City of the Dead (Cairo)', 'Coptic Cairo', 'Daher, Egypt', 'Downtown Cairo', 'El Manial', 'El Marg', 'El Matareya, Cairo', 'El Qobbah', 'El Rehab', 'El Sahel', 'El Sakkakini', 'Ezbet El Haggana', 'Ezbet El Nakhl', 'Faggala', 'Fifth Settlement', 'Fustat', 'Garden City, Cairo', 'Gezira (Cairo)', 'Heliopolis, Cairo', 'Islamic Cairo', 'Maadi', 'Old Cairo', 'Roda Island', 'Shubra (administrative region)', 'Shubra', 'Shubra El Kheima', 'Wagh El Birket', 'Zamalek', 'Zeitoun, Cairo']


In [9]:
# south sinia district
giza_url='https://en.wikipedia.org/wiki/South_Sinai_Governorate'
page_1=requests.get(url=giza_url).text

In [10]:
soup_si=BeautifulSoup(page_1)

In [11]:
part_si=soup_si.find('table',attrs={"class":"wikitable"})

In [12]:
data_si=part_si.findAll('td')

In [13]:
south_sina_dist=[]
for i in range(0,len(data_si),5):
    south_sina_dist.append(data_si[i].text)

In [14]:
south_sina_dist

['Abu Radis (incl. Abu Zenima)',
 'El Tor',
 'Dahab',
 'Nuweiba',
 'Ras Sidr',
 'Saint Catherine',
 'Sharm El Sheikh 1',
 'Sharm El Sheikh 2',
 'Taba']

In [15]:
geo_locator=Nominatim(user_agent='abdo')

In [16]:
all_data=cairo_dist+south_sina_dist

In [37]:
len(south_sina_dist)

9

In [17]:
df=pd.DataFrame(all_data,columns=['reg'])

In [18]:
df.reg.replace('\(.*|\)','',regex=True,inplace=True)

In [19]:
df.reg=df.reg+' ,egypt'

In [20]:
df.loc[0,'reg']+=',cairo'

In [21]:
df.head()

,reg
0,"Abbassia ,egypt,cairo"
1,"Ain Shams ,egypt"
2,"Azbakeya ,egypt"
3,"Bab al-Louq ,egypt"
4,"Boulaq ,egypt"


In [22]:
output=df.copy()
for i in df.itertuples():
    try:
        location=geo_locator.geocode(f"{i.reg}")
        output.loc[output.reg==i.reg,['lat','long']]=location.latitude,location.longitude
        
    except:
        output.loc[output.reg==i.reg,['lat','long']]=None,None
        print(f"{i.reg}  : Erorr")
    else:
        
        print(f"{i.reg}  :  {location}")
        time.sleep(2)

Abbassia ,egypt,cairo  : Erorr
Ain Shams ,egypt  :  عين شمس, القاهرة, محافظة القاهرة, 113311, مصر
Azbakeya ,egypt  :  الأزبكية, القاهرة, محافظة القاهرة, 11574, مصر
Bab al-Louq ,egypt  :  سوق خضار باب اللوق, شارع الفلكي, الدواوين, باب اللوق, القاهرة, محافظة القاهرة, 11513, مصر
Boulaq ,egypt  :  مستشفي بولاق ابو العلا, شارع شركس السلمانيه, سوق العصر, السبتية, القاهرة, محافظة القاهرة, 11221, مصر
City of the Dead  ,egypt  : Erorr
Coptic Cairo ,egypt  :  القاهرة القبطية, شارع الصباح, مصر القديمة, القاهرة, محافظة القاهرة, 11632, مصر
Daher, Egypt ,egypt  : Erorr
Downtown Cairo ,egypt  :  وسط البلد, شارع وسط البلد, القبه, حدائق القبة, القاهرة, محافظة القاهرة, 11663, مصر
El Manial ,egypt  :  منيل شيحا, 12912, مصر
El Marg ,egypt  :  مرج, القليوبية, 11833, مصر
El Matareya, Cairo ,egypt  :  المطريه القبليه, القاهرة, محافظة القاهرة, مصر
El Qobbah ,egypt  : Erorr
El Rehab ,egypt  :  الرحاب, محافظة القاهرة, 11841, مصر
El Sahel ,egypt  :  الساحل, القاهرة, محافظة القاهرة, مصر
El Sakkakini ,egypt  : Ero

In [23]:
#count nan values that  throw error while searching geocode
len(output.loc[output.lat.isna()])

7

In [24]:
#removes errors
output.dropna(inplace=True)

In [25]:
#see a duplicated lat or long 
output.loc[output.long.duplicated()]

,reg,lat,long
29,"Shubra ,egypt",30.128599,31.242218
30,"Shubra El Kheima ,egypt",30.128599,31.242218
41,"Sharm El Sheikh 2 ,egypt",27.864442,34.295447


In [26]:
#remove duplicated
output.drop_duplicates(subset='long',inplace=True)

In [27]:
#Egypt coordiante 26.8206° N, 30.8025° E
loc_egy=[26.8206,30.025]
egy_map=folium.Map(loc_egy,zoom_start=6)

In [28]:
egy_map

In [29]:
for cell in output.itertuples():
    folium.CircleMarker(location=[cell.lat,cell.long],redias=5,popup=cell.reg,color='blue',
                       fill=True,fill_color='red',fill_opacity=.7).add_to(egy_map)

In [30]:
egy_map

In [51]:
output.to_csv("reg_df.csv",index=False)

In [44]:
output.loc[output.reg.replace(' ,egypt','',regex=True).isin(south_sina_dist),'prov']='south_sina'

In [47]:
output.fillna('cairo',inplace=True)

In [50]:
output.loc[34,'prov']='south_sina'